# 宿題2
一対他法で課題を行った

In [1]:
import csv
import numpy as np

## 訓練データ読み込み

In [3]:
train_filenames = [
    'digit/digit_train0.csv',
    'digit/digit_train1.csv',
    'digit/digit_train2.csv',
    'digit/digit_train3.csv',
    'digit/digit_train4.csv',
    'digit/digit_train5.csv',
    'digit/digit_train6.csv',
    'digit/digit_train7.csv',
    'digit/digit_train8.csv',
    'digit/digit_train9.csv'
]
train_digits = []
for train_filename in train_filenames:
    with open(train_filename) as f:
        reader = csv.reader(f)
        l = np.array([list(map(float, row)) for row in reader])
        train_digits.append(l)

In [4]:
train_digits = np.array(train_digits).reshape(-1, 256)
train_digits.shape

(5000, 256)

## 訓練

$$\hat{\theta} =  K^{-1} y$$を計算($K$はカーネル行列)

各ラベルの訓練において、$y$だけがラベルで変わるので$K^{-1}$を一度だけ計算している

In [7]:
def kernel(x0, x1):
    return np.exp(-0.5*(np.linalg.norm(x0-x1, ord=2))/h**2)
def train_all(train_digits):
    thetas = []
    n = len(train_digits)
    K = np.array(
        [
            [
                kernel(train_digits[i], train_digits[j]) for j in range(n)
            ]
            for i in range(n)
        ]
    )
    K_inv = np.linalg.inv(K)
    print("first matrix compute finished", flush=True)
    for i in range(10):
        y = np.ones([n, 1])
        y *= -1
        y[i*500:(i+1)*500] = 1
        theta = K_inv @ y
        thetas.append(theta)
        print(f"{i} th train finished", flush=True)
    return thetas

In [8]:
h = 1
classifiers = train_all(train_digits)

first matrix compute finished
5000 (5000, 5000) (5000, 1)
0 th train finished
5000 (5000, 5000) (5000, 1)
1 th train finished
5000 (5000, 5000) (5000, 1)
2 th train finished
5000 (5000, 5000) (5000, 1)
3 th train finished
5000 (5000, 5000) (5000, 1)
4 th train finished
5000 (5000, 5000) (5000, 1)
5 th train finished
5000 (5000, 5000) (5000, 1)
6 th train finished
5000 (5000, 5000) (5000, 1)
7 th train finished
5000 (5000, 5000) (5000, 1)
8 th train finished
5000 (5000, 5000) (5000, 1)
9 th train finished


In [9]:
classifiers[0].shape

(5000, 1)

## テスト
各数字に対してテストデータで検証を行った

In [14]:
test_filenames = [
    'digit/digit_test0.csv',
    'digit/digit_test1.csv',
    'digit/digit_test2.csv',
    'digit/digit_test3.csv',
    'digit/digit_test4.csv',
    'digit/digit_test5.csv',
    'digit/digit_test6.csv',
    'digit/digit_test7.csv',
    'digit/digit_test8.csv',
    'digit/digit_test9.csv'
]

def classify(data, train_digits):
    l = np.zeros(10)
    for i in range(10):
        l[i] = sum([classifiers[i][j]*kernel(data, train_digits[j]) for j in range(len(train_digits))])
    return np.argmax(l)

result = np.zeros((10, 10))
d = 0
for test_filename in test_filenames:
    with open(test_filename) as f:
        reader = csv.reader(f)
        l = np.array([list(map(float, row)) for row in reader])
    for i in range(len(l)):
        est = classify(l[i], train_digits)
        result[d, est] += 1
    d += 1

## 結果
縦が正解のカテゴリで、横が予測したカテゴリ

In [16]:
print(result)

[[196.   0.   1.   2.   0.   0.   1.   0.   0.   0.]
 [  0. 199.   1.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0. 191.   0.   5.   0.   0.   1.   3.   0.]
 [  0.   0.   1. 194.   0.   3.   0.   1.   1.   0.]
 [  0.   1.   1.   0. 193.   0.   3.   0.   0.   2.]
 [  2.   0.   1.   4.   2. 188.   0.   0.   1.   2.]
 [  1.   0.   2.   0.   1.   1. 195.   0.   0.   0.]
 [  0.   0.   2.   0.   4.   0.   0. 190.   1.   3.]
 [  0.   0.   0.   4.   0.   4.   0.   0. 190.   2.]
 [  0.   0.   0.   0.   5.   0.   0.   1.   1. 193.]]


In [22]:
accuracy = sum(np.diag(result))/sum(sum(result))
print(f"accuracy = {accuracy}")

accuracy = 0.9645
